In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader
import math
import os
import torch
from tensorflow.keras import Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report



In [40]:

def search_csv_files(directory,activities_of_interest = None):
    csv_files = []
    for current_folder, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                if activities_of_interest is not None:
                    for activity in activities_of_interest:
                        if activity in file:
                            full_path = os.path.join(current_folder, file)
                            csv_files.append(full_path)
                            break
                else:
                    full_path = os.path.join(current_folder, file)
                    csv_files.append(full_path)
                    break
    return csv_files




falls_files = search_csv_files(directory='output', activities_of_interest=['Activity13', 'Activity14', 'Activity15'])
adl_files = search_csv_files(directory='output', activities_of_interest=['Activity1', 'Activity2', 'Activity3', 'Activity4', 'Activity5', 'Activity6', 'Activity7', 'Activity8', 'Activity9', 'Activity10', 'Activity11', 'Activity12'])




File: output/Subject15/Activity14/Trial1/UMAFALL_Subject15Activity14Trial1.csv, Shape: (289, 11)
File: output/Subject15/Activity14/Trial4/UMAFALL_Subject15Activity14Trial4.csv, Shape: (267, 11)
File: output/Subject15/Activity14/Trial3/UMAFALL_Subject15Activity14Trial3.csv, Shape: (300, 11)
File: output/Subject15/Activity14/Trial2/UMAFALL_Subject15Activity14Trial2.csv, Shape: (300, 11)
File: output/Subject15/Activity13/Trial4/UMAFALL_Subject15Activity13Trial4.csv, Shape: (300, 11)
File: output/Subject15/Activity13/Trial3/UMAFALL_Subject15Activity13Trial3.csv, Shape: (299, 11)
File: output/Subject15/Activity13/Trial2/UMAFALL_Subject15Activity13Trial2.csv, Shape: (300, 11)
File: output/Subject15/Activity15/Trial1/UMAFALL_Subject15Activity15Trial1.csv, Shape: (300, 11)
File: output/Subject15/Activity15/Trial4/UMAFALL_Subject15Activity15Trial4.csv, Shape: (300, 11)
File: output/Subject15/Activity15/Trial3/UMAFALL_Subject15Activity15Trial3.csv, Shape: (300, 11)
File: output/Subject15/Activit

In [41]:
def load_and_pad(file, use_gyro=False, max_len=300):
    # Read the CSV file
    df = pd.read_csv(file)
    
    # Define the columns to use
    if use_gyro:
        cols = ['Accelerometer: x-axis (g)', 'Accelerometer: y-axis (g)', 'Accelerometer: z-axis (g)',
                'Gyroscope: x-axis (rad/s)', 'Gyroscope: y-axis (rad/s)', 'Gyroscope: z-axis (rad/s)']
    else:
        cols = ['Accelerometer: x-axis (g)', 'Accelerometer: y-axis (g)', 'Accelerometer: z-axis (g)']
    
    # Clean and convert data for each column
    for col in cols:
        if df[col].dtype == 'object':  # If column has string values
            # Replace incorrect number format (assuming European format with '.' as thousand separator)
            df[col] = df[col].astype(str).str.replace('.', '', n=1)  # Remove first period (thousand separator)
            df[col] = pd.to_numeric(df[col].str.replace(',', '.'), errors='coerce')  # Replace comma with period for decimal

    # Extract features and convert to numpy array
    features = df[cols].to_numpy().astype('float32')
    
    # Pad or truncate as needed
    if features.shape[0] > max_len:
        features = features[:max_len, :]
    elif features.shape[0] < max_len:
        pad = np.zeros((max_len - features.shape[0], features.shape[1]), dtype='float32')
        features = np.vstack((features, pad))
    
    return features




def build_dataset(falls_files, adl_files, max_len=300):
    X_acc, X_all, y = [], [], []

    for file in falls_files:
        X_acc.append(load_and_pad(file, use_gyro=False, max_len=max_len))
        X_all.append(load_and_pad(file, use_gyro=True, max_len=max_len))
        y.append(1)

    for file in adl_files:
        X_acc.append(load_and_pad(file, use_gyro=False, max_len=max_len))
        X_all.append(load_and_pad(file, use_gyro=True, max_len=max_len))
        y.append(0)

    return np.array(X_acc), np.array(X_all), np.array(y)



In [42]:
X_acc, X_all, y = build_dataset(falls_files, adl_files)


print("Accelerometer only:", X_acc.shape)
print("Accelerometer + Gyroscope:", X_all.shape)
print("Labels:", y.shape)

Accelerometer only: (876, 300, 3)
Accelerometer + Gyroscope: (876, 300, 6)
Labels: (876,)


In [ ]:
USE_GYRO = False

X = X_all if USE_GYRO else X_acc

y = y  # already 0 (ADL) or 1 (Fall)

 
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model = Sequential([
    Input(shape=(300, X.shape[2])),
    Conv1D(filters=32, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {acc:.2f}")

# Report
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7103 - loss: nan - val_accuracy: 0.7214 - val_loss: nan
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7819 - loss: nan - val_accuracy: 0.7214 - val_loss: nan
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7840 - loss: nan - val_accuracy: 0.7214 - val_loss: nan
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7974 - loss: nan - val_accuracy: 0.7214 - val_loss: nan
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7594 - loss: nan - val_accuracy: 0.7214 - val_loss: nan
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7919 - loss: nan - val_accuracy: 0.7214 - val_loss: nan
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7828 - loss: nan - val_accuracy: 0.7214 - val_loss: nan
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8016 - loss: nan - val_accuracy: 0.7214 - val_loss: nan
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s

/Users/amir/Documents/projects/fall-detection/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/amir/Documents/projects/fall-detection/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/amir/Documents/projects/fall-detection/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  